In [1]:
import pandas as pd
import numpy as np
import catboost as cb

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import impute
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [2]:
np.random.seed(0) 

train_dat = pd.read_csv("data/train.csv.nosync.csv")
test_dat_with_id = pd.read_csv("data/test.csv.nosync.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [3]:
print(train_dat.shape)
print(test_dat_with_id.shape)
print(sample_submission.shape)

(5380, 767)
(4403, 766)
(4403, 2)


In [4]:
train_dat = train_dat.drop('id', axis=1)
test_dat = test_dat_with_id.drop('id', axis=1)

In [5]:
print(train_dat.shape)
print(test_dat.shape)

(5380, 766)
(4403, 765)


In [6]:
train_missing_values = train_dat.isnull().sum()
test_missing_values = test_dat.isnull().sum()
print(train_missing_values)
print(test_missing_values)

x001     10
x002      0
x003      3
x004      0
x005     10
       ... 
x762     53
x763      0
x764      3
x765    375
y         0
Length: 766, dtype: int64
x001     23
x002      0
x003      4
x004      0
x005     23
       ... 
x761      0
x762     46
x763      0
x764      4
x765    272
Length: 765, dtype: int64


In [7]:
train_dat_imputation = train_dat.drop('y', axis = 1)

In [8]:
scaler = StandardScaler()
train_scaled_data = pd.DataFrame(scaler.fit_transform(train_dat_imputation))
train_imputer = impute.KNNImputer(n_neighbors=5, weights="uniform")
imputed_scaled_train = train_imputer.fit_transform(train_scaled_data)
unscaled_train = scaler.inverse_transform(imputed_scaled_train)
train_imputed = pd.DataFrame(unscaled_train, columns=train_dat_imputation.columns)

scaler = StandardScaler()
test_scaled_data = pd.DataFrame(scaler.fit_transform(test_dat))
test_imputer = impute.KNNImputer(n_neighbors=5, weights="uniform")
imputed_scaled_test = test_imputer.fit_transform(test_scaled_data)
unscaled_test = scaler.inverse_transform(imputed_scaled_test)
test_imputed = pd.DataFrame(unscaled_test, columns=test_dat.columns)

In [9]:
train_missing_values = train_imputed.isnull().sum()
test_missing_values = test_imputed.isnull().sum()
print(train_missing_values)
print(test_missing_values)

x001    0
x002    0
x003    0
x004    0
x005    0
       ..
x761    0
x762    0
x763    0
x764    0
x765    0
Length: 765, dtype: int64
x001    0
x002    0
x003    0
x004    0
x005    0
       ..
x761    0
x762    0
x763    0
x764    0
x765    0
Length: 765, dtype: int64


In [10]:
X_train = train_imputed
X_test = test_imputed
y_train = train_dat['y']

In [11]:
pca = PCA(n_components = 80, svd_solver = 'full')
X_train = pca.fit_transform(X_train)
print(len(pca.explained_variance_ratio_))

80


In [12]:
X_test = pca.transform(X_test)
print(len(pca.explained_variance_ratio_))

80


In [13]:
y_train = np.log(y_train)

In [14]:
model = cb.CatBoostRegressor(loss_function = 'RMSE')

In [15]:
parameters = {'depth': [4,6,8,10],
              'learning_rate':[0.05, 0.1, 0.2],
              'iterations': [50, 100, 150],
              'l2_leaf_reg': [0.2, 0.5, 1, 3]
             }

grid = GridSearchCV(estimator = model, param_grid = parameters, cv = 3, n_jobs=-1, scoring = 'neg_root_mean_squared_error')
grid_search = grid.fit(X_train, y_train)

0:	learn: 0.9954650	total: 95.2ms	remaining: 4.67s
1:	learn: 0.9900941	total: 108ms	remaining: 2.58s
2:	learn: 0.9845602	total: 120ms	remaining: 1.88s
3:	learn: 0.9806079	total: 131ms	remaining: 1.51s
4:	learn: 0.9749887	total: 144ms	remaining: 1.3s
5:	learn: 0.9709902	total: 168ms	remaining: 1.23s
6:	learn: 0.9666143	total: 180ms	remaining: 1.11s
7:	learn: 0.9635105	total: 194ms	remaining: 1.02s
8:	learn: 0.9601621	total: 210ms	remaining: 956ms
9:	learn: 0.9568063	total: 225ms	remaining: 901ms
10:	learn: 0.9536862	total: 242ms	remaining: 858ms
11:	learn: 0.9501795	total: 251ms	remaining: 796ms
12:	learn: 0.9475836	total: 265ms	remaining: 753ms
13:	learn: 0.9444505	total: 277ms	remaining: 713ms
14:	learn: 0.9421072	total: 288ms	remaining: 672ms
15:	learn: 0.9392075	total: 297ms	remaining: 632ms
16:	learn: 0.9364845	total: 309ms	remaining: 600ms
17:	learn: 0.9342965	total: 323ms	remaining: 574ms
18:	learn: 0.9316674	total: 331ms	remaining: 541ms
19:	learn: 0.9297721	total: 342ms	remaini

0:	learn: 0.9784727	total: 93.7ms	remaining: 4.59s
1:	learn: 0.9624627	total: 107ms	remaining: 2.56s
2:	learn: 0.9514673	total: 118ms	remaining: 1.85s
3:	learn: 0.9397002	total: 128ms	remaining: 1.47s
4:	learn: 0.9285958	total: 138ms	remaining: 1.25s
5:	learn: 0.9219475	total: 151ms	remaining: 1.11s
6:	learn: 0.9138130	total: 166ms	remaining: 1.02s
7:	learn: 0.9101757	total: 179ms	remaining: 938ms
8:	learn: 0.9065980	total: 195ms	remaining: 887ms
9:	learn: 0.9013044	total: 207ms	remaining: 829ms
10:	learn: 0.8986403	total: 220ms	remaining: 780ms
11:	learn: 0.8952906	total: 237ms	remaining: 751ms
12:	learn: 0.8915941	total: 251ms	remaining: 714ms
13:	learn: 0.8879419	total: 264ms	remaining: 679ms
14:	learn: 0.8833834	total: 274ms	remaining: 639ms
15:	learn: 0.8794041	total: 282ms	remaining: 599ms
16:	learn: 0.8766047	total: 294ms	remaining: 570ms
17:	learn: 0.8741931	total: 303ms	remaining: 538ms
18:	learn: 0.8717369	total: 315ms	remaining: 513ms
19:	learn: 0.8693294	total: 324ms	remain

20:	learn: 0.9279470	total: 322ms	remaining: 1.21s
21:	learn: 0.9261814	total: 335ms	remaining: 1.19s
22:	learn: 0.9237098	total: 350ms	remaining: 1.17s
23:	learn: 0.9217867	total: 361ms	remaining: 1.14s
24:	learn: 0.9199718	total: 376ms	remaining: 1.13s
25:	learn: 0.9177703	total: 388ms	remaining: 1.1s
26:	learn: 0.9157587	total: 400ms	remaining: 1.08s
27:	learn: 0.9141620	total: 412ms	remaining: 1.06s
28:	learn: 0.9127846	total: 425ms	remaining: 1.04s
29:	learn: 0.9113863	total: 439ms	remaining: 1.02s
30:	learn: 0.9100798	total: 453ms	remaining: 1.01s
31:	learn: 0.9084987	total: 466ms	remaining: 989ms
32:	learn: 0.9072686	total: 484ms	remaining: 982ms
33:	learn: 0.9059287	total: 497ms	remaining: 964ms
34:	learn: 0.9048866	total: 510ms	remaining: 947ms
35:	learn: 0.9030076	total: 524ms	remaining: 932ms
36:	learn: 0.9018715	total: 538ms	remaining: 916ms
37:	learn: 0.9001648	total: 552ms	remaining: 901ms
38:	learn: 0.8984249	total: 571ms	remaining: 893ms
39:	learn: 0.8973918	total: 585m

21:	learn: 0.9249734	total: 325ms	remaining: 1.15s
22:	learn: 0.9232276	total: 334ms	remaining: 1.12s
23:	learn: 0.9214292	total: 355ms	remaining: 1.12s
24:	learn: 0.9195294	total: 370ms	remaining: 1.11s
25:	learn: 0.9177407	total: 384ms	remaining: 1.09s
26:	learn: 0.9161814	total: 403ms	remaining: 1.09s
27:	learn: 0.9145305	total: 418ms	remaining: 1.07s
28:	learn: 0.9132393	total: 435ms	remaining: 1.06s
29:	learn: 0.9122469	total: 444ms	remaining: 1.04s
30:	learn: 0.9108366	total: 462ms	remaining: 1.03s
31:	learn: 0.9093913	total: 478ms	remaining: 1.01s
32:	learn: 0.9081122	total: 490ms	remaining: 995ms
33:	learn: 0.9070126	total: 510ms	remaining: 990ms
34:	learn: 0.9060636	total: 521ms	remaining: 967ms
35:	learn: 0.9047486	total: 533ms	remaining: 948ms
36:	learn: 0.9038531	total: 547ms	remaining: 931ms
37:	learn: 0.9024582	total: 560ms	remaining: 914ms
38:	learn: 0.9013480	total: 572ms	remaining: 895ms
39:	learn: 0.9002132	total: 586ms	remaining: 879ms
40:	learn: 0.8991422	total: 605

70:	learn: 0.7570811	total: 1.07s	remaining: 437ms
71:	learn: 0.7548440	total: 1.09s	remaining: 423ms
72:	learn: 0.7531766	total: 1.1s	remaining: 409ms
73:	learn: 0.7518681	total: 1.13s	remaining: 396ms
74:	learn: 0.7502184	total: 1.14s	remaining: 381ms
75:	learn: 0.7484918	total: 1.15s	remaining: 365ms
76:	learn: 0.7472453	total: 1.16s	remaining: 348ms
77:	learn: 0.7450025	total: 1.18s	remaining: 332ms
78:	learn: 0.7436463	total: 1.19s	remaining: 317ms
79:	learn: 0.7416478	total: 1.2s	remaining: 301ms
80:	learn: 0.7398577	total: 1.22s	remaining: 285ms
81:	learn: 0.7376582	total: 1.23s	remaining: 270ms
82:	learn: 0.7357177	total: 1.24s	remaining: 254ms
83:	learn: 0.7337046	total: 1.26s	remaining: 239ms
84:	learn: 0.7321641	total: 1.27s	remaining: 224ms
85:	learn: 0.7305036	total: 1.28s	remaining: 209ms
86:	learn: 0.7292574	total: 1.29s	remaining: 193ms
87:	learn: 0.7274993	total: 1.31s	remaining: 178ms
88:	learn: 0.7259977	total: 1.32s	remaining: 163ms
89:	learn: 0.7241112	total: 1.33s

...

20:	learn: 0.4932494	total: 12.7s	remaining: 1m 18s
21:	learn: 0.4833117	total: 13.2s	remaining: 1m 16s
22:	learn: 0.4703273	total: 13.7s	remaining: 1m 15s
23:	learn: 0.4587216	total: 14.2s	remaining: 1m 14s
24:	learn: 0.4497374	total: 14.8s	remaining: 1m 13s
25:	learn: 0.4424834	total: 15.2s	remaining: 1m 12s
26:	learn: 0.4270155	total: 15.9s	remaining: 1m 12s
27:	learn: 0.4110818	total: 16.4s	remaining: 1m 11s
28:	learn: 0.4058325	total: 17.1s	remaining: 1m 11s
29:	learn: 0.3929104	total: 17.5s	remaining: 1m 10s
30:	learn: 0.3821279	total: 18.1s	remaining: 1m 9s
31:	learn: 0.3723465	total: 18.6s	remaining: 1m 8s
32:	learn: 0.3595960	total: 19.1s	remaining: 1m 7s
33:	learn: 0.3505544	total: 19.6s	remaining: 1m 7s
34:	learn: 0.3420049	total: 20.2s	remaining: 1m 6s
35:	learn: 0.3355353	total: 20.7s	remaining: 1m 5s
36:	learn: 0.3255801	total: 21.1s	remaining: 1m 4s
37:	learn: 0.3168022	total: 21.6s	remaining: 1m 3s
38:	learn: 0.3111256	total: 22.1s	remaining: 1m 3s
39:	learn: 0.3026337	

16:	learn: 0.8141514	total: 9.17s	remaining: 1m 11s
17:	learn: 0.8063848	total: 9.7s	remaining: 1m 11s
18:	learn: 0.7999457	total: 10.2s	remaining: 1m 10s
19:	learn: 0.7923151	total: 10.7s	remaining: 1m 9s
20:	learn: 0.7848991	total: 11.2s	remaining: 1m 8s
21:	learn: 0.7784750	total: 11.7s	remaining: 1m 7s
22:	learn: 0.7720462	total: 12.1s	remaining: 1m 7s
23:	learn: 0.7657333	total: 12.7s	remaining: 1m 6s
24:	learn: 0.7588982	total: 13.1s	remaining: 1m 5s
25:	learn: 0.7525491	total: 13.7s	remaining: 1m 5s
26:	learn: 0.7465982	total: 14.2s	remaining: 1m 4s
27:	learn: 0.7415000	total: 14.7s	remaining: 1m 4s
28:	learn: 0.7361742	total: 15.3s	remaining: 1m 3s
29:	learn: 0.7303214	total: 15.8s	remaining: 1m 3s
30:	learn: 0.7239149	total: 16.3s	remaining: 1m 2s
31:	learn: 0.7190109	total: 17s	remaining: 1m 2s
32:	learn: 0.7135531	total: 17.5s	remaining: 1m 2s
33:	learn: 0.7081015	total: 18.1s	remaining: 1m 1s
34:	learn: 0.7023769	total: 18.7s	remaining: 1m 1s
35:	learn: 0.6983514	total: 19.

19:	learn: 0.4980323	total: 10.7s	remaining: 1m 9s
20:	learn: 0.4821148	total: 11.2s	remaining: 1m 8s
21:	learn: 0.4741985	total: 11.7s	remaining: 1m 7s
22:	learn: 0.4591442	total: 12.2s	remaining: 1m 7s
23:	learn: 0.4512881	total: 12.7s	remaining: 1m 6s
24:	learn: 0.4432203	total: 13.2s	remaining: 1m 5s
25:	learn: 0.4336877	total: 13.6s	remaining: 1m 4s
26:	learn: 0.4213718	total: 14.2s	remaining: 1m 4s
27:	learn: 0.4062091	total: 14.6s	remaining: 1m 3s
28:	learn: 0.3961443	total: 15.2s	remaining: 1m 3s
29:	learn: 0.3862073	total: 15.7s	remaining: 1m 2s
30:	learn: 0.3784248	total: 16.3s	remaining: 1m 2s
31:	learn: 0.3657692	total: 16.8s	remaining: 1m 1s
32:	learn: 0.3573366	total: 17.5s	remaining: 1m 1s
33:	learn: 0.3524536	total: 18s	remaining: 1m 1s
34:	learn: 0.3419707	total: 18.7s	remaining: 1m 1s
35:	learn: 0.3337050	total: 19.2s	remaining: 1m
36:	learn: 0.3267787	total: 19.8s	remaining: 1m
37:	learn: 0.3204871	total: 20.4s	remaining: 1m
38:	learn: 0.3110129	total: 21s	remaining:

19:	learn: 0.8031781	total: 10.9s	remaining: 1m 11s
20:	learn: 0.7951434	total: 11.4s	remaining: 1m 10s
21:	learn: 0.7875636	total: 11.9s	remaining: 1m 9s
22:	learn: 0.7797649	total: 12.4s	remaining: 1m 8s
23:	learn: 0.7747093	total: 12.9s	remaining: 1m 7s
24:	learn: 0.7682349	total: 13.4s	remaining: 1m 7s
25:	learn: 0.7623628	total: 13.9s	remaining: 1m 6s
26:	learn: 0.7563770	total: 14.5s	remaining: 1m 5s
27:	learn: 0.7520816	total: 15s	remaining: 1m 5s
28:	learn: 0.7458979	total: 15.6s	remaining: 1m 4s
29:	learn: 0.7404568	total: 16.2s	remaining: 1m 4s
30:	learn: 0.7324403	total: 16.8s	remaining: 1m 4s
31:	learn: 0.7268141	total: 17.4s	remaining: 1m 4s
32:	learn: 0.7173285	total: 18s	remaining: 1m 3s
33:	learn: 0.7115050	total: 18.7s	remaining: 1m 3s
34:	learn: 0.7049384	total: 19.4s	remaining: 1m 3s
35:	learn: 0.7012001	total: 20s	remaining: 1m 3s
36:	learn: 0.6957248	total: 20.5s	remaining: 1m 2s
37:	learn: 0.6910817	total: 21s	remaining: 1m 2s
38:	learn: 0.6871664	total: 21.7s	rem

0:	learn: 0.9878193	total: 57.7ms	remaining: 8.6s
1:	learn: 0.9778982	total: 63.3ms	remaining: 4.68s
2:	learn: 0.9702779	total: 69.7ms	remaining: 3.42s
3:	learn: 0.9622367	total: 74.1ms	remaining: 2.71s
4:	learn: 0.9539025	total: 78.6ms	remaining: 2.28s
5:	learn: 0.9483282	total: 82.6ms	remaining: 1.98s
6:	learn: 0.9423130	total: 87.2ms	remaining: 1.78s
7:	learn: 0.9379413	total: 91.9ms	remaining: 1.63s
8:	learn: 0.9342342	total: 96.5ms	remaining: 1.51s
9:	learn: 0.9302142	total: 100ms	remaining: 1.4s
10:	learn: 0.9273887	total: 105ms	remaining: 1.32s
11:	learn: 0.9243862	total: 112ms	remaining: 1.28s
12:	learn: 0.9211548	total: 116ms	remaining: 1.22s
13:	learn: 0.9188012	total: 121ms	remaining: 1.17s
14:	learn: 0.9151020	total: 125ms	remaining: 1.12s
15:	learn: 0.9121856	total: 129ms	remaining: 1.08s
16:	learn: 0.9095146	total: 134ms	remaining: 1.05s
17:	learn: 0.9079111	total: 138ms	remaining: 1.01s
18:	learn: 0.9057841	total: 142ms	remaining: 979ms
19:	learn: 0.9035340	total: 146ms	

In [16]:
print(grid_search.best_score_)
print(grid_search.best_params_)

-0.8783116326165009
{'depth': 4, 'iterations': 150, 'l2_leaf_reg': 1, 'learning_rate': 0.1}


In [17]:
best_depth = grid_search.best_params_['depth']
best_iterations = grid_search.best_params_['iterations']
best_learning_rate = grid_search.best_params_['learning_rate']
best_leaf_reg = grid_search.best_params_['l2_leaf_reg']

model = cb.CatBoostRegressor(loss_function = 'RMSE', 
                             depth = best_depth, 
                             iterations = best_iterations, 
                             learning_rate = best_learning_rate,
                             l2_leaf_reg = best_leaf_reg)

In [18]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

y_pred = np.exp(y_pred)

submission_dat = {'id': test_dat_with_id['id'], 'y': y_pred}
submission = pd.DataFrame(data=submission_dat)
submission.shape

0:	learn: 0.9878193	total: 5.22ms	remaining: 777ms
1:	learn: 0.9778982	total: 8.89ms	remaining: 658ms
2:	learn: 0.9702779	total: 11.8ms	remaining: 577ms
3:	learn: 0.9622367	total: 14ms	remaining: 512ms
4:	learn: 0.9539025	total: 16.2ms	remaining: 470ms
5:	learn: 0.9483282	total: 18.6ms	remaining: 447ms
6:	learn: 0.9423130	total: 20.8ms	remaining: 425ms
7:	learn: 0.9379413	total: 23.3ms	remaining: 414ms
8:	learn: 0.9342342	total: 26.2ms	remaining: 411ms
9:	learn: 0.9302142	total: 28.4ms	remaining: 397ms
10:	learn: 0.9273887	total: 30.4ms	remaining: 385ms
11:	learn: 0.9243862	total: 32.6ms	remaining: 375ms
12:	learn: 0.9211548	total: 35ms	remaining: 368ms
13:	learn: 0.9188012	total: 37.2ms	remaining: 361ms
14:	learn: 0.9151020	total: 39.3ms	remaining: 354ms
15:	learn: 0.9121856	total: 41.4ms	remaining: 347ms
16:	learn: 0.9095146	total: 43.7ms	remaining: 342ms
17:	learn: 0.9079111	total: 45.8ms	remaining: 336ms
18:	learn: 0.9057841	total: 47.9ms	remaining: 330ms
19:	learn: 0.9035340	total

(4403, 2)

In [19]:
submission.to_csv('model.csv', index=False)

38:	learn: 0.5561249	total: 23.4s	remaining: 1m 6s
39:	learn: 0.5492495	total: 23.9s	remaining: 1m 5s
40:	learn: 0.5437205	total: 24.5s	remaining: 1m 5s
41:	learn: 0.5383049	total: 25s	remaining: 1m 4s
42:	learn: 0.5339827	total: 25.6s	remaining: 1m 3s
43:	learn: 0.5307263	total: 26.2s	remaining: 1m 3s
44:	learn: 0.5251503	total: 26.8s	remaining: 1m 2s
45:	learn: 0.5188334	total: 27.4s	remaining: 1m 2s
46:	learn: 0.5127753	total: 28.1s	remaining: 1m 1s
47:	learn: 0.5059457	total: 28.7s	remaining: 1m 1s
48:	learn: 0.5019011	total: 29.3s	remaining: 1m
49:	learn: 0.4963338	total: 30s	remaining: 60s
50:	learn: 0.4922945	total: 30.6s	remaining: 59.4s
51:	learn: 0.4880377	total: 31.2s	remaining: 58.8s
52:	learn: 0.4817095	total: 31.8s	remaining: 58.2s
53:	learn: 0.4775382	total: 32.5s	remaining: 57.7s
54:	learn: 0.4713156	total: 33s	remaining: 57s
55:	learn: 0.4658688	total: 33.7s	remaining: 56.5s
56:	learn: 0.4613560	total: 34.3s	remaining: 56s
57:	learn: 0.4550159	total: 34.9s	remaining: 5

37:	learn: 0.7208110	total: 22.4s	remaining: 1m 6s
38:	learn: 0.7176646	total: 22.9s	remaining: 1m 5s
39:	learn: 0.7143381	total: 23.5s	remaining: 1m 4s
40:	learn: 0.7092553	total: 24s	remaining: 1m 3s
41:	learn: 0.7055936	total: 24.6s	remaining: 1m 3s
42:	learn: 0.7013684	total: 25.2s	remaining: 1m 2s
43:	learn: 0.6959518	total: 25.7s	remaining: 1m 1s
44:	learn: 0.6919720	total: 26.3s	remaining: 1m 1s
45:	learn: 0.6869522	total: 27s	remaining: 1m 1s
46:	learn: 0.6828634	total: 27.6s	remaining: 1m
47:	learn: 0.6779185	total: 28.1s	remaining: 59.8s
48:	learn: 0.6733256	total: 28.8s	remaining: 59.4s
49:	learn: 0.6692048	total: 29.5s	remaining: 58.9s
50:	learn: 0.6657298	total: 30.1s	remaining: 58.4s
51:	learn: 0.6627285	total: 30.7s	remaining: 57.9s
52:	learn: 0.6564174	total: 31.4s	remaining: 57.4s
53:	learn: 0.6524652	total: 32s	remaining: 56.9s
54:	learn: 0.6477655	total: 32.7s	remaining: 56.5s
55:	learn: 0.6443914	total: 33.3s	remaining: 55.9s
56:	learn: 0.6389050	total: 33.9s	remain

37:	learn: 0.5681894	total: 22.6s	remaining: 1m 6s
38:	learn: 0.5624778	total: 23.2s	remaining: 1m 5s
39:	learn: 0.5569096	total: 23.8s	remaining: 1m 5s
40:	learn: 0.5517482	total: 24.3s	remaining: 1m 4s
41:	learn: 0.5472351	total: 24.9s	remaining: 1m 4s
42:	learn: 0.5411453	total: 25.6s	remaining: 1m 3s
43:	learn: 0.5366920	total: 26.2s	remaining: 1m 3s
44:	learn: 0.5302959	total: 26.7s	remaining: 1m 2s
45:	learn: 0.5229554	total: 27.4s	remaining: 1m 1s
46:	learn: 0.5171901	total: 28.1s	remaining: 1m 1s
47:	learn: 0.5099496	total: 28.7s	remaining: 1m 1s
48:	learn: 0.5040063	total: 29.4s	remaining: 1m
49:	learn: 0.4985011	total: 30s	remaining: 1m
50:	learn: 0.4946685	total: 30.7s	remaining: 59.5s
51:	learn: 0.4898019	total: 31.3s	remaining: 59s
52:	learn: 0.4854552	total: 31.9s	remaining: 58.3s
53:	learn: 0.4801455	total: 32.5s	remaining: 57.8s
54:	learn: 0.4731586	total: 33.1s	remaining: 57.2s
55:	learn: 0.4676730	total: 33.8s	remaining: 56.7s
56:	learn: 0.4618031	total: 34.4s	remaini

37:	learn: 0.5695269	total: 22.7s	remaining: 1m 7s
38:	learn: 0.5636833	total: 23.3s	remaining: 1m 6s
39:	learn: 0.5586175	total: 23.9s	remaining: 1m 5s
40:	learn: 0.5537835	total: 24.4s	remaining: 1m 4s
41:	learn: 0.5480615	total: 25s	remaining: 1m 4s
42:	learn: 0.5427262	total: 25.7s	remaining: 1m 3s
43:	learn: 0.5361001	total: 26.3s	remaining: 1m 3s
44:	learn: 0.5279770	total: 26.9s	remaining: 1m 2s
45:	learn: 0.5205482	total: 27.6s	remaining: 1m 2s
46:	learn: 0.5156700	total: 28.2s	remaining: 1m 1s
47:	learn: 0.5065936	total: 28.8s	remaining: 1m 1s
48:	learn: 0.5008217	total: 29.5s	remaining: 1m
49:	learn: 0.4953690	total: 30.2s	remaining: 1m
50:	learn: 0.4905685	total: 30.8s	remaining: 59.8s
51:	learn: 0.4839496	total: 31.4s	remaining: 59.2s
52:	learn: 0.4749834	total: 32s	remaining: 58.6s
53:	learn: 0.4692950	total: 32.7s	remaining: 58.1s
54:	learn: 0.4628528	total: 33.3s	remaining: 57.4s
55:	learn: 0.4574328	total: 33.9s	remaining: 56.9s
56:	learn: 0.4515542	total: 34.6s	remaini

37:	learn: 0.3835231	total: 19.8s	remaining: 58.5s
38:	learn: 0.3777531	total: 20.4s	remaining: 58.1s
39:	learn: 0.3699658	total: 20.9s	remaining: 57.6s
40:	learn: 0.3636261	total: 21.5s	remaining: 57s
41:	learn: 0.3572348	total: 22s	remaining: 56.5s
42:	learn: 0.3472595	total: 22.6s	remaining: 56.2s
43:	learn: 0.3384467	total: 23.1s	remaining: 55.7s
44:	learn: 0.3319996	total: 23.7s	remaining: 55.3s
45:	learn: 0.3287750	total: 24.3s	remaining: 54.8s
46:	learn: 0.3244366	total: 24.8s	remaining: 54.3s
47:	learn: 0.3185275	total: 25.4s	remaining: 53.9s
48:	learn: 0.3114344	total: 25.9s	remaining: 53.4s
49:	learn: 0.3053816	total: 26.5s	remaining: 53s
50:	learn: 0.2954713	total: 27.1s	remaining: 52.6s
51:	learn: 0.2877928	total: 27.7s	remaining: 52.2s
52:	learn: 0.2805390	total: 28.3s	remaining: 51.7s
53:	learn: 0.2748760	total: 28.8s	remaining: 51.2s
54:	learn: 0.2684888	total: 29.3s	remaining: 50.6s
55:	learn: 0.2646542	total: 29.9s	remaining: 50.1s
56:	learn: 0.2576535	total: 30.4s	rem

34:	learn: 0.4088403	total: 18.7s	remaining: 1m 1s
35:	learn: 0.4026339	total: 19.2s	remaining: 1m
36:	learn: 0.3938015	total: 19.6s	remaining: 60s
37:	learn: 0.3853746	total: 20.3s	remaining: 59.7s
38:	learn: 0.3746329	total: 20.8s	remaining: 59.2s
39:	learn: 0.3630187	total: 21.4s	remaining: 58.7s
40:	learn: 0.3586027	total: 22s	remaining: 58.4s
41:	learn: 0.3509318	total: 22.6s	remaining: 58.1s
42:	learn: 0.3435308	total: 23.2s	remaining: 57.6s
43:	learn: 0.3351698	total: 23.8s	remaining: 57.3s
44:	learn: 0.3288264	total: 24.3s	remaining: 56.6s
45:	learn: 0.3196643	total: 24.9s	remaining: 56.3s
46:	learn: 0.3147820	total: 25.4s	remaining: 55.7s
47:	learn: 0.3087220	total: 26.1s	remaining: 55.4s
48:	learn: 0.3042029	total: 26.6s	remaining: 54.8s
49:	learn: 0.2973468	total: 27.2s	remaining: 54.4s
50:	learn: 0.2920926	total: 27.8s	remaining: 54s
51:	learn: 0.2847383	total: 28.4s	remaining: 53.4s
52:	learn: 0.2790175	total: 28.9s	remaining: 52.9s
53:	learn: 0.2751584	total: 29.4s	remain

38:	learn: 0.3837920	total: 20.9s	remaining: 59.6s
39:	learn: 0.3774040	total: 21.6s	remaining: 59.3s
40:	learn: 0.3693902	total: 22.1s	remaining: 58.7s
41:	learn: 0.3618045	total: 22.7s	remaining: 58.4s
42:	learn: 0.3534801	total: 23.2s	remaining: 57.8s
43:	learn: 0.3466186	total: 23.7s	remaining: 57.2s
44:	learn: 0.3385789	total: 24.3s	remaining: 56.7s
45:	learn: 0.3289633	total: 24.9s	remaining: 56.2s
46:	learn: 0.3204213	total: 25.4s	remaining: 55.8s
47:	learn: 0.3157727	total: 26.1s	remaining: 55.5s
48:	learn: 0.3061668	total: 26.7s	remaining: 55.1s
49:	learn: 0.2990405	total: 27.3s	remaining: 54.5s
50:	learn: 0.2892955	total: 27.8s	remaining: 53.9s
51:	learn: 0.2855513	total: 28.3s	remaining: 53.3s
52:	learn: 0.2789157	total: 28.8s	remaining: 52.7s
53:	learn: 0.2738012	total: 29.3s	remaining: 52.1s
54:	learn: 0.2686577	total: 29.8s	remaining: 51.5s
55:	learn: 0.2616081	total: 30.3s	remaining: 50.9s
56:	learn: 0.2564942	total: 30.9s	remaining: 50.3s
57:	learn: 0.2523694	total: 31.

37:	learn: 0.7669538	total: 20s	remaining: 59.1s
38:	learn: 0.7632048	total: 20.6s	remaining: 58.6s
39:	learn: 0.7600952	total: 21.2s	remaining: 58.2s
40:	learn: 0.7573377	total: 21.7s	remaining: 57.7s
41:	learn: 0.7534915	total: 22.3s	remaining: 57.3s
42:	learn: 0.7505127	total: 22.8s	remaining: 56.8s
43:	learn: 0.7465172	total: 23.4s	remaining: 56.4s
44:	learn: 0.7426196	total: 24s	remaining: 55.9s
45:	learn: 0.7394015	total: 24.5s	remaining: 55.4s
46:	learn: 0.7363605	total: 25.1s	remaining: 54.9s
47:	learn: 0.7332898	total: 25.7s	remaining: 54.6s
48:	learn: 0.7304994	total: 26.2s	remaining: 54.1s
49:	learn: 0.7262546	total: 26.7s	remaining: 53.5s
50:	learn: 0.7234070	total: 27.3s	remaining: 52.9s
51:	learn: 0.7202860	total: 27.8s	remaining: 52.4s
52:	learn: 0.7173222	total: 28.3s	remaining: 51.8s
53:	learn: 0.7135431	total: 28.8s	remaining: 51.2s
54:	learn: 0.7103402	total: 29.4s	remaining: 50.7s
55:	learn: 0.7066757	total: 29.9s	remaining: 50.1s
56:	learn: 0.7039932	total: 30.4s	r